#####################################
##    뉴스 요약 트랜스포머 모델   ##
#####################################

0. 인코딩 타입 지정, 라이브러리 임포트, GPU 확인

In [274]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.callbacks import Callback, ModelCheckpoint
from keras.models import load_model
import pickle

gpus = tf.config.list_physical_devices('GPU')
print("Available GPUs:", gpus)

Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


1. 전처리 데이터 로드

In [275]:
with open('D:/TJ_FInal_Project/KDJ/News_Summarization/Data/pickleTemp/sentences.pkl', 'rb') as f:
    sentences = pickle.load(f)

with open('D:/TJ_FInal_Project/KDJ/News_Summarization/Data/pickleTemp/abs.pkl', 'rb') as f:
    abs = pickle.load(f)

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs':sentences,
        'dec_inputs':abs[:, :-1]
    },
    {
        'outputs':abs[:, 1:]
    }
))

print(f'sentence : {sentences[1]}')
print(f'\nabs : {abs[1]}')

sentence : [15540 15340  1978 ...     0     0     0]

abs : [15540  5028  8327   603 10997   510 11850     3  4828   165 13700    17
  5791  2139 13177   165  9496  6258  3176   709    10  3639 11254   666
  3198   409  5176  1010 11475 15316 13183 12924     8    57  1121    10
 12635  2023   803   661   846 13087  3849   911   102  2928  9170 15330
 15541     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0    

2. 하이퍼파라미터 설정

In [276]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.load_from_file('D:/TJ_FInal_Project/KDJ/News_Summarization/Data/문서요약 텍스트/Preprocess/tokenizerTemp')

SEN_MAX_LENGTH = 1199
ABS_MAX_LENGTH = 199

VOCAB_SIZE = tokenizer.vocab_size + 2
BATCH_SIZE = 8
BUFFER_SIZE = 20000

D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.5

EPOCHS = 2000

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

3. 트랜스포머 모델 빌드

3-1. 포지셔널 인코딩

In [277]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    sines = tf.math.sin(angle_rads[:, 0::2])

    cosines = tf.math.cos(angle_rads[:, 1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

3-2. 패딩 마스크

In [278]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  return mask[:, tf.newaxis, tf.newaxis, :]

3-3. 룩-어헤드 마스크

In [279]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

3-4. 셀프 어텐션(스케일드 닷 프로덕트 어텐션)

In [280]:
def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  if mask is not None:
    logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(logits, axis=-1)

  output = tf.matmul(attention_weights, value)

  return output, attention_weights

3-5. 멀티헤드 어텐션

In [281]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    outputs = self.dense(concat_attention)

    return outputs

3-6. 인코더 층

In [282]:
def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs, 'key': inputs, 'value': inputs,
          'mask': padding_mask
      })

  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

3-7. 인코더

In [283]:
def encoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

3-8. 디코더 층

In [284]:
def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")

  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs, 'key': inputs, 'value': inputs,
          'mask': look_ahead_mask
      })

  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1, 'key': enc_outputs, 'value': enc_outputs,
          'mask': padding_mask
      })

  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

3-9. 디코더

In [285]:
def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)
  
tf.keras.backend.clear_session()

3-10. 트랜스포머

In [286]:
def transformer(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)
  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[inputs, enc_padding_mask]) 

  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)
  
  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

3-11. 모델 구조 선언

In [287]:
model = transformer(
  vocab_size=VOCAB_SIZE,
  num_layers=NUM_LAYERS,
  dff=DFF,
  d_model=D_MODEL,
  num_heads=NUM_HEADS,
  dropout=DROPOUT
)

(1, 15542, 256)
(1, 15542, 256)


4. 커스텀 스케줄러

In [288]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
  
learning_rate = CustomSchedule(D_MODEL)


5. 오차 함수 및 옵티마이저

In [289]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, ABS_MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

optimizer = tf.keras.optimizers.Adam(
  learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9
)

6. 정확도 계산 함수 및 컴파일

In [290]:
def acuuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, ABS_MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[acuuracy])

7. 검증 함수

In [291]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size+1]

def evaluate(sentence):
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(ABS_MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

8. 예측 함수

In [292]:
def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])
  print(f'원문 : {sentence}')
  print(f'\n요약 : {predicted_sentence}')

  return predicted_sentence

9. 검증 데이터 준비 및 epoch 마다 검증

In [293]:
df = pd.read_csv('D:/TJ_FInal_Project/KDJ/News_Summarization/Data/문서요약 텍스트/Preprocess/finalPreprocess.csv')
sentence = df['sentence']
abs = df['abs']

previous_loss = tf.Variable(float('inf'), trainable=False)

class EpochValidation(Callback):
    def on_epoch_end(self, epoch, logs=None):
        self.epochPrint(epoch, logs)

    def epochPrint(self, epoch, logs=None):
        if epoch % 10 == 0:
            print("\n***************  첫번째 예측  ***************")
            predict(sentence[100])
            print(f'\n정답 : {abs[100]}')
            print("\n***************  두번째 예측  ***************")
            predict(sentence[200])
            print(f'\n정답 : {abs[200]}')
        
        current_loss = logs.get('loss')
        
        if current_loss is not None:
            if current_loss < previous_loss.numpy():
                model.save_weights('D:/TJ_FInal_Project/KDJ/News_Summarization/Model/transformer.h5')
                print(f'\n손실 값 감소! 이전 손실: {previous_loss.numpy()}, 현재 손실: {current_loss}')
            else:
                print(f'\n손실 값 증가 또는 동일')
            
            previous_loss.assign(current_loss)

10. 학습 및 모델 저장

In [294]:
model.fit(dataset, epochs=EPOCHS, callbacks=[EpochValidation()]) #,checkpoint
# model.load_weights('D:/TJ_FInal_Project/KDJ/News_Summarization/Model/trasformer.h5')

Epoch 1/2000
125/125 [==============================] - ETA: 0s - loss: 1.6666 - acuuracy: 0.0023
***************  첫번째 예측  ***************

원문 : 연구 교육 전시 활용 목적 백운산지키키협의회 서울대 불통 여전 반발 김영신 기자 서울대 남부학술림 산림교육연구센터가 지난 23일 3년여의 공사를 끝내고 옥룡면 추산시험장내에 문을 열었다. 이날 준공식에는 성낙인 서울대학교 총장을 비롯한 광양, 순천, 구례 지자체대표와 지역주민, 시민단체 등 100여명이 참석했다. 서울대 남부학술림은 총사업비 64억여원을 들여 남부학술림의 우수한 산림자원을 활용한 연구와 교육, 산림과학 인재양성, 국내외 학술행사 개최, 다양한 사회교육과 산림문화 기회제공 등을 목적으로 준공됐다. 서울대 남부학술림 산림교육연구센터는 연면적 867평의 지상 3층 철근 콘크리트와 목구조 건물로 대강당, 강의실, 세미나실, 연구실, 행정실 등의 시설을 갖췄다. 박정호 임장은 새로 준공한 산림교육연구센터와 추산생활관은 교육과 연구를 목적으로 할 경우 누구나 이용할 수 있다 며 작품전, 음악회, 전시회 등이 가능하다. 백운산지키기협의회는 23일 성명을 발표하고 서울대의 끝없는 지역과의 불통에 대해 심히 유감을 표명한다 며 강력 반발했다. 협의회는 서울대가 70년간 백운산 학술림을 관리 운영하면서 연구한 논문은 약 55편으로 1년에 1편 미만 이라며 이 정도 실적이면 백운산을 법인서울대학교에 무상으로 양도하는 것 보다는 산림청 산하 국유림으로 관리하게 하는 것이 바람직하다 고 강조했다. 특히 산림교육연구센터 건립 과정에서 지역과 어떠한 소통도 없었다고 비판했다. 협의회는 서울대는 실질적으로 시민의 마음을 얻기 위해 낮은 자세로 다가서야 한다 면서 백운산 무상양도와 지역과의 소통, 협의회와 총장 면담 을 촉구했다.

요약 : .

정답 : 서울대 남부학술림 산림교육연구센터가 옥룡면 추산시험장내에 완공되었

KeyboardInterrupt: 